# Benchmarks

Contains benchmark simulations with other models. This was outsourced to google
colab to make it accessible to the whole community. For the applications, it requires access the the data, change the path accordingly to data in your drive or simply upload the data to your runtime. 

Note, the notebook was run on cpu but depending on your data you might want to 
use a GPU. This might change the value of the benchmarks slightly

In [1]:
%%capture
!pip install torch torchtuples pycox lifelines matplotlib

In [2]:
# Import packages (need both tensorflow and torch)
import pickle

# Wrangling
import numpy as np
import pandas as pd
from scipy.stats import gamma
import re
from collections import namedtuple
from sklearn.preprocessing import StandardScaler

# Backends
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import torchtuples as tt

# Models and metrics
from pycox.evaluation import EvalSurv
from pycox.models import LogisticHazard, PMF, DeepHitSingle, MTLR

In [3]:
def calculate_preds(output_raw, y_surv_, y_cens_):
    output_ = (pd.DataFrame(list(map(np.ravel, output_raw)))
                                .reset_index()
                                .rename(columns={'index':'timeline'}))

    time_idx = np.array(output_['timeline'].astype(float))
    output_ = output_.set_index(time_idx).drop(columns='timeline')

    evaluation_results = EvalSurv(output_,
                                    y_surv_,
                                    y_cens_,
                                    censor_surv='km')
    
    linspace_brier = np.linspace(y_surv_.min(), 
                                 y_surv_.max(), 
                                 100)
    
    
    concordance_idx = evaluation_results.concordance_td('antolini')
    brier_idx = evaluation_results.integrated_brier_score(linspace_brier) 

    return concordance_idx, brier_idx

In [4]:
class DeepKaplanMeier:
    """
    Deep-Kaplan-Meier estimator class.


    Parameters
        ----------
        periods: int
            Total number of periods that observation has taken place

        weights_max: float, optional (default=0)
            Maximum add to weights for losses. This parameter is optional 
            and can be used in the case heavy censoring is present. In turn
            it will give more weight to losses corresponding to later periods

        learning_rate: float, optional (default=0.005)
            The learning rate used in the optimization
    """
    def __init__(self,
                 periods=None, 
                 weights_max = 0, 
                 learning_rate=0.005):
        """
        ToDo: Make period counter more flexible (include between period counters)
        """
        self._set_periods(periods)
        self.weights_max = weights_max
        self.learning_rate = learning_rate
        
    def _set_periods(self,
                     periods):
        assert isinstance(periods, int), f"""Specify the periods as integer,
                                             you specified it as {type(periods)}"""
        self.periods = periods

    def prepare_survival(self, survival_time, censoring):
        """
        Prepares target input matrix for estimation
        
        Parameters
        ----------
        survival time: int
            Indicator of last observed period, either censoring time or
            failure time
            
        censoring: bool
            Censoring indicator, if True observation is censored
            
        returns: y-matrix and weights-matrix
        """
        if isinstance(survival_time, list):
            observations = len(survival_time)
        elif isinstance(survival_time, np.ndarray):
            observations = survival_time.shape[0]
        else:
            raise Exception(f"""you need to provide either an array or
                                list of survival times,
                                you provided {type(survival_time)}""")
                
        weight_matrix = np.ones(shape=(observations, self.periods+1))
        survival_matrix = pad_sequences( [np.repeat(1, self.periods + 1)] + 
                                         [np.repeat(1, surv_time+1) if cens_ else
                                         np.repeat(1, surv_time) for
                                         surv_time, cens_ in zip(survival_time, censoring)],
                                        padding='post')[1:,:]

        weight_matrix[censoring,:] = weight_matrix[censoring,:] * survival_matrix[censoring,:]

        return survival_matrix, weight_matrix

    def create_output_struct(self, 
                             hidden_layer_output,
                             activation,
                             preoutput_units):
        """
        ToDo: make static (set periods in input rather than from class)
        Helper function to create connected output units according to 
        a given parametrization. Main function is compile_model
        """
        outputs = []
        if preoutput_units is not None:
            for _ in range(self.periods + 1):
                aggregation_layer = tf.keras.layers.Dense(units=preoutput_units, 
                                                        activation=activation)(hidden_layer_output)
                output_unit = tf.keras.layers.Dense(1, activation='sigmoid')(aggregation_layer)
                outputs.append(output_unit)
        else:
            for _ in range(self.periods + 1):
                output_unit = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer_output)
                outputs.append(output_unit)

        for unit_idx in np.arange(1, self.periods + 1):
            outputs[unit_idx] = tf.keras.layers.Multiply()([outputs[unit_idx], outputs[unit_idx - 1]])
        
        return outputs

    @staticmethod
    def create_rep_struct(input_mod,
                          hidden_units,
                          activation):
        """
        Helper function to create embedding layer for the hidden
        features. Main function is compile_model
        """

        if isinstance(hidden_units, int):
            input_mod = tf.keras.layers.Dense(hidden_units, 
                              activation=activation)(input_mod)
        elif isinstance(hidden_units, list):
            for units_ in hidden_units:
                input_mod = tf.keras.layers.Dense(units_, 
                              activation=activation)(input_mod)  
        else:
            raise TypeError('hidden_units must be int or list or ints')

        return input_mod

    def compile_model(self,
                      input_shape,
                      hidden_units,
                      activation='relu',
                      preoutput=50):
        """
        Compiles keras model according to inputs

        Parameters
        ----------
        input_shape: tuple
            Tuple specifying input shape. In general of the form (k, )

        hidden_units: int or list(int)
            If integer, the number of hidden units in the single hidden layer
            If list, one integers specifying hidden units per hidden layer

        activation: str, optional (default='relu')
            activation function for hidden units

        preoutput: int, optional (default=50)
            Number of hidden units in the final layer when combining the weights
            from the embedding layer
        """
        tf.keras.backend.clear_session()

        input_layer = tf.keras.Input(shape=input_shape)
        input_mod = input_layer

        input_layer = tf.keras.Input(shape=input_shape)
        input_mod = input_layer

        input_mod = self.create_rep_struct(input_mod, hidden_units, activation)
        
        outputs = self.create_output_struct(hidden_layer_output=input_mod,
                                            activation=activation,
                                            preoutput_units=preoutput)

        self.model = tf.keras.Model(inputs=input_layer, outputs=outputs)
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                           loss="binary_crossentropy",
                           loss_weights=np.array(1 + np.linspace(0,
                                                self.weights_max,36))
                           )

    def return_comp_graph_raw(self,
                              input_shape,
                              hidden_units,
                              activation='relu', 
                              preoutput=None):
        """
        Returns computational graph by combining the embedding structure
        and the output structure

        Parameters
        ----------
        input_shape: tuple
            Tuple specifying input shape. In general of the form (k, )

        hidden_units: int or list(int)
            If integer, the number of hidden units in the single hidden layer
            If list, one integers specifying hidden units per hidden layer

        activation: str, optional (Default: 'relu')
            activation function for hidden units

        preoutput: int, optional (Default: 50)
            Number of hidden units in the final layer when combining the weights
            from the embedding layer


        returns:
            uncompiled model structure as a tf.keras.Model
        """
        tf.keras.backend.clear_session()
        input_layer = tf.keras.Input(shape=input_shape)
        input_mod = input_layer

        input_mod = self.create_rep_struct(input_mod, hidden_units, activation)
        
        outputs = self.create_output_struct(hidden_layer_output=input_mod,
                                            activation=activation, 
                                            preoutput_units=preoutput)
                                            
        model_raw =  tf.keras.Model(inputs=input_layer, outputs=outputs)

        return model_raw

    def fit_model(self, X_train, y_survival, censoring,
                  epochs=50, batch_size = 256, verbose=False):
        """
        Fits the compiled model using data

        Parameters
        ----------
        X_train: np.array or tf.tensor
            array or similar of the predictions variables, must be of the 
            dimension as specified with "input_shape" otherwise an error
            will be raised

        y_survival: np.array of int
            the survival times for every row in X_train, must be specified
            as an integer at the moment. Otherwise it must be specified during
            the preprocessing

        censoring: np.array of bool
            vector indicating whether an observation was censored or not,
            used to create the weighting matrix

        epochs: int, optional (default=50)
            number of epochs (passes of the training set) to be performed until
            stop

        batch_size: int, optional (default=256)
            batch size for training operation

        verbose: bool, optional (default=False)
            whether to show training progress for every batch that is processed
            if set to True will also return a history object that can be used
            for plotting
        """
        y_train_np, weights_train_np = self.prepare_survival(y_survival, censoring)
        
        y_train = [y_train_np[:, i] for i in range(y_train_np.shape[1])]
        weights_train = [weights_train_np[:, i] for i in range(weights_train_np.shape[1])]
        
        if verbose:
            hist_ = self.model.fit(X_train,
                                   y_train,
                                   sample_weight=weights_train,
                                   epochs=epochs,
                                   batch_size=batch_size)
            return hist_
        else:
            self.model.fit(x=X_train,
                           y=y_train,
                           sample_weight=weights_train,
                           verbose=verbose,
                           epochs=epochs,
                           batch_size=batch_size)

## Applications 


### Application Medical

In [5]:
df = pd.read_csv('/content/drive/MyDrive/data_colab/application_training_smart.csv')
y_target = np.array(df.tevent.astype(int))
y_censoring = np.where(df.event == 1, False, True)

df_val = pd.read_csv('/content/drive/MyDrive/data_colab/application_test_smart.csv')
y_target_val = np.array(df_val.tevent.astype(int))
y_censoring_val = np.where(df_val.event == 1, False, True)
y_censoring_val_eval = np.where(df_val.event == 1, 1, 0)

In [6]:
total_periods = int(y_target.max())

In [7]:
features_unscaled = df.loc[:, df.columns[3:]]
features_unscaled_val = df_val.loc[:, df_val.columns[3:]]

features_numeric = features_unscaled.max(axis=0) > 1

numeric_ = features_unscaled.loc[:,features_numeric.values]
binary_ = features_unscaled.loc[:,~features_numeric.values].to_numpy()

numeric_val = features_unscaled_val.loc[:,features_numeric.values]
binary_val = features_unscaled_val.loc[:,~features_numeric.values].to_numpy()

scaler = StandardScaler()
scaler.fit(numeric_)

numeric_trans = scaler.transform(numeric_)
numeric_trans_val = scaler.transform(numeric_val)

features = np.concatenate([numeric_trans, binary_], axis=1).astype('float32')
features_val = np.concatenate([numeric_trans_val, binary_val], axis=1).astype('float32')

In [8]:
# Instantiate model
tf.keras.utils.set_random_seed(123)
model_smart = DeepKaplanMeier(total_periods,
                              learning_rate=0.05,
                              weights_max=3.2)

model_smart.compile_model((30, ),
                          [18,6,8],
                          preoutput=6)

for repeater_epoch in range(5):
  print(f'...this is epoch {5*repeater_epoch+5}')

  tf.keras.utils.set_random_seed(123)
  model_smart.fit_model(features,
                      y_target.astype(int), 
                      y_censoring,
                      epochs=5,
                      verbose=False, 
                      batch_size=128)

...this is epoch 5
...this is epoch 10
...this is epoch 15
...this is epoch 20
...this is epoch 25


In [9]:
preds_raw = model_smart.model.predict(features_val)

linspace_brier = np.linspace(y_target_val.min(), y_target_val.max(), 100)
score_conc, score_brier = calculate_preds(preds_raw, y_target_val, y_censoring_val_eval)

print(f'concordance: {score_conc} \nbrier: {score_brier}')

25/25 [==============================] - 1s 4ms/step
concordance: 0.6868355646525617 
brier: 0.1216432980045617


In [10]:
preds_raw = model_smart.model.predict(features_val)

linspace_brier = np.linspace(y_target_val.min(), y_target_val.max(), 100)
score_conc, score_brier = calculate_preds(preds_raw, y_target_val, y_censoring_val_eval)

print(f'concordance: {score_conc} \nbrier: {score_brier}')

25/25 [==============================] - 0s 4ms/step
concordance: 0.6868355646525617 
brier: 0.1216432980045617


In [11]:
# Specific model architectures
from pycox.models import LogisticHazard, PMF, DeepHitSingle, MTLR
from pycox.evaluation import EvalSurv

_ = torch.manual_seed(123)

In [12]:
# reload as need different parametrization
df = pd.read_csv('/content/drive/MyDrive/data_colab/application_training_smart.csv')
y_target = np.array(df.tevent.astype(int))
y_censoring = np.where(df.event == 1, 1, 0)

df_val = pd.read_csv('/content/drive/MyDrive/data_colab/application_test_smart.csv')
y_target_val = np.array(df_val.tevent.astype(int))
y_censoring_val = np.where(df_val.event == 1, 1, 0)

In [13]:
total_periods = y_target.max() + 1

features_unscaled = df.loc[:, df.columns[3:]]
features_unscaled_val = df_val.loc[:, df_val.columns[3:]]

features_numeric = features_unscaled.max(axis=0) > 1

numeric_ = features_unscaled.loc[:,features_numeric.values]
binary_ = features_unscaled.loc[:,~features_numeric.values].to_numpy()

numeric_val = features_unscaled_val.loc[:,features_numeric.values]
binary_val = features_unscaled_val.loc[:,~features_numeric.values].to_numpy()

scaler = StandardScaler()
scaler.fit(numeric_)

numeric_trans = scaler.transform(numeric_)
numeric_trans_val = scaler.transform(numeric_val)

features = np.concatenate([numeric_trans, binary_], axis=1).astype('float32')
features_val = np.concatenate([numeric_trans_val, binary_val], axis=1).astype('float32')

In [14]:
%%capture
_ = torch.manual_seed(123) 
all_results_smart = {
    'LogisticHazard': {},
    'PMF': {},
    'DeepHitSingle': {},
    'MTLR': {},}

for mod_ in [LogisticHazard, PMF, DeepHitSingle, MTLR]: 
    str_pattern = r"([^\.]*)$"
    model_key = re.search(str_pattern, str(mod_)).group(0)[:-2]

    labtrans = mod_.label_transform(total_periods)
    y_train_ = labtrans.fit_transform(*(y_target, y_censoring))

    in_features = features.shape[1]
    out_features = labtrans.out_features

    net = torch.nn.Sequential(
        torch.nn.Linear(in_features, 4),
        torch.nn.ReLU(),

        torch.nn.Linear(4, 16),
        torch.nn.ReLU(),

        torch.nn.Linear(16, 20),
        torch.nn.ReLU(),

        torch.nn.Linear(20, 20),
        torch.nn.ReLU(),

        torch.nn.Linear(20, out_features)
    )
    
    model = mod_(net,
                tt.optim.Adam(0.005),
                duration_index=labtrans.cuts)

    batch_size = 128
    epochs = 25

    log = model.fit(features, y_train_, batch_size, epochs) 
    surv = model.predict_surv_df(features_val)
    
    time_grid = np.linspace(y_target_val.min(), y_target_val.max(), 100)

    evaluation_results = EvalSurv(surv,
                                    y_target_val,
                                    y_censoring_val,
                                    censor_surv='km')
    concordance_idx = evaluation_results.concordance_td('antolini')
    brier_idx = evaluation_results.integrated_brier_score(time_grid) 

    all_results_smart[model_key]['model'] = model
    all_results_smart[model_key]['predictions'] = surv
    all_results_smart[model_key]['concordance'] = concordance_idx
    all_results_smart[model_key]['brier'] = brier_idx

In [15]:
for model_key in all_results_smart.keys():
    print(model_key, np.round(all_results_smart[model_key]['concordance'],3))

LogisticHazard 0.63
PMF 0.662
DeepHitSingle 0.664
MTLR 0.677


In [16]:
for model_key in all_results_smart.keys():
    print(model_key, np.round(all_results_smart[model_key]['brier'],3))

LogisticHazard 0.112
PMF 0.116
DeepHitSingle 0.109
MTLR 0.112


### Economic application 

In [17]:
df = pd.read_csv('/content/drive/MyDrive/data_colab/data_econ_train.csv')
y_target = np.array(df.time.astype(int))
y_censoring = np.where(df.event == 1, False, True)
y_censoring_target_eval = np.where(df.event == 1, 1, 0)

df_val = pd.read_csv('/content/drive/MyDrive/data_colab/data_econ_test.csv')
y_target_val = np.array(df_val.time.astype(int))
y_censoring_val = np.where(df_val.event == 1, False, True)
y_censoring_val_eval = np.where(df_val.event == 1, 1, 0)

In [18]:
total_periods = int(y_target.max())

In [19]:
# Scale features
numeric_ = df.loc[:, df.columns[4:-1]]
binary_ = np.where(df.fac_ui == 1, 1, 0).reshape(-1,1)

numeric_val = df_val.loc[:, df_val.columns[4:-1]]
binary_val = np.where(df_val.fac_ui == 1, 1, 0).reshape(-1,1)

scaler = StandardScaler()
scaler.fit(numeric_)

numeric_trans = scaler.transform(numeric_)
numeric_trans_val = scaler.transform(numeric_val)

features = np.concatenate([numeric_trans, binary_], axis=1).astype('float32')
features_val = np.concatenate([numeric_trans_val, binary_val], axis=1).astype('float32')

In [20]:
# Instantiate model
tf.keras.utils.set_random_seed(123)
model_smart = DeepKaplanMeier(total_periods,
                              learning_rate=0.05,
                              weights_max=1.6)

model_smart.compile_model((6, ),
                          [14, 12, 8],
                          preoutput=20)

for repeater_epoch in range(5):
  print(f'...this is epoch {5*repeater_epoch+5}')

  tf.keras.utils.set_random_seed(123)
  model_smart.fit_model(features,
                      y_target.astype(int), 
                      y_censoring,
                      epochs=5,
                      verbose=False, 
                      batch_size=256)
  
preds_raw = model_smart.model.predict(features_val)
score_concordance, score_brier = calculate_preds(preds_raw, y_target_val, y_censoring_val_eval)

...this is epoch 5
...this is epoch 10
...this is epoch 15
...this is epoch 20
...this is epoch 25
21/21 [==============================] - 1s 4ms/step


In [21]:
print(f'concordance: {score_concordance} \nbrier: {score_brier}')

concordance: 0.7145001667963972 
brier: 0.16716137251218152


In [22]:
%%capture
_ = torch.manual_seed(123) 
all_results_econ = {
    'LogisticHazard': {},
    'PMF': {},
    'DeepHitSingle': {},
    'MTLR': {},}

for mod_ in [LogisticHazard, PMF, DeepHitSingle, MTLR]: 
    str_pattern = r"([^\.]*)$"
    model_key = re.search(str_pattern, str(mod_)).group(0)[:-2]

    labtrans = mod_.label_transform(total_periods)
    y_train_ = labtrans.fit_transform(*(y_target, y_censoring_target_eval))

    in_features = features.shape[1]
    out_features = labtrans.out_features

    net = torch.nn.Sequential(
        torch.nn.Linear(in_features, 14),
        torch.nn.ReLU(),

        torch.nn.Linear(14, 12),
        torch.nn.ReLU(),

        torch.nn.Linear(12, 8),
        torch.nn.ReLU(),

        torch.nn.Linear(8, 20),
        torch.nn.ReLU(),

        torch.nn.Linear(20, out_features)
    )
    
    model = mod_(net,
                tt.optim.Adam(0.005),
                duration_index=labtrans.cuts)

    batch_size = 256
    epochs = 50

    log = model.fit(features, y_train_, batch_size, epochs) 
    surv = model.predict_surv_df(features_val)
    
    time_grid = np.linspace(y_target_val.min(), y_target_val.max(), 100)

    evaluation_results = EvalSurv(surv,
                                    y_target_val,
                                    y_censoring_val_eval,
                                    censor_surv='km')
    concordance_idx = evaluation_results.concordance_td('antolini')
    brier_idx = evaluation_results.integrated_brier_score(time_grid) 

    all_results_econ[model_key]['model'] = model
    all_results_econ[model_key]['predictions'] = surv
    all_results_econ[model_key]['concordance'] = concordance_idx
    all_results_econ[model_key]['brier'] = brier_idx

In [23]:
for model_key in all_results_econ.keys():
    print(model_key, np.round(all_results_econ[model_key]['concordance'],3))

LogisticHazard 0.708
PMF 0.493
DeepHitSingle 0.691
MTLR 0.544


In [24]:
for model_key in all_results_econ.keys():
    print(model_key, np.round(all_results_econ[model_key]['brier'],3))

LogisticHazard 0.162
PMF 0.167
DeepHitSingle 0.179
MTLR 0.172


## Simulations (selected)

In [25]:
def generate_nonlinear_data(size, seed):
    size_ = size
    seed_ = seed

    np.random.seed(seed_)
    x_1 = np.random.exponential(0.1, size=size_)
    x_2 = np.random.normal(10, np.sqrt(5), size=size_)
    x_3 = np.random.poisson(5, size=size_)

    features = np.array([x_1, x_2, x_3]).T

    # Arrange Targets
    survial_model = np.sin(x_1)*3.14 + 0.318*x_2 + 2.72*np.abs(np.cos(x_3))

    true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
    ceiling_surv_ = np.ceil(true_survival_time)

    censoring_transformed = np.minimum(2*(survial_model/survial_model.max()),1)

    censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
    censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

    observed_survival = ceiling_surv_.copy()
    observed_survival[censored_instances] = censored_surv

    variable_dict = dict({'true_survival': ceiling_surv_,
                        'observed_survival': observed_survival, 
                        'censoring': censored_instances,
                        'features' :  features})

    return variable_dict

def generate_heavy_censored_data(size, seed):
    size_ = size
    seed_ = seed

    np.random.seed(seed_)
    x_1 = np.random.exponential(0.1, size=size_)
    x_2 = np.random.normal(10, np.sqrt(5), size=size_)
    x_3 = np.random.poisson(5, size=size_)

    features = np.array([x_1, x_2, x_3]).T

    # Arrange Targets
    survial_model = -3.14*x_1 + 0.318*x_2 + 2.72*x_3

    true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
    ceiling_surv_ = true_survival_time

    censoring_transformed = np.minimum(0.75*(survial_model/survial_model.max()),1)

    censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
    censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

    observed_survival = ceiling_surv_.copy()
    observed_survival[censored_instances] = censored_surv

    variable_dict = dict({'true_survival': ceiling_surv_,
                        'observed_survival': observed_survival, 
                        'censoring': censored_instances,
                        'features' :  features})

    return variable_dict


In [26]:
# Nonlinear
data_train_nonlin = generate_nonlinear_data(size=3000, seed=42)
data_test_nonlin = generate_nonlinear_data(size=1000, seed=123)

Nonlinear = namedtuple('Nonlinear', "train test")
nonlinear_data = Nonlinear(data_train_nonlin, data_test_nonlin)

# Heavy censoring
data_train_heavy_cens = generate_heavy_censored_data(size=3000, seed=42)
data_test_heavy_cens = generate_heavy_censored_data(size=1000, seed=123)
data_train_heavy_cens_large = generate_heavy_censored_data(size=12000, seed=42)

HeavyCensoring = namedtuple('HeavyCensoring', "train trainlarge test")
heavy_censored_data = HeavyCensoring(data_train_heavy_cens,
                                     data_train_heavy_cens_large,
                                     data_test_heavy_cens)

In [27]:
%%capture
_ = torch.manual_seed(123) # still some randomness from the GPU!
all_results = {
    'LogisticHazard': {},
    'PMF': {},
    'DeepHitSingle': {},
    'MTLR': {},}

for dataset_ in [nonlinear_data, heavy_censored_data]:
    dataset_key = type(dataset_).__name__
    print(dataset_key)
    
    scaler_ = StandardScaler()
    scaler_.fit(dataset_.train['features'])

    scaled_features_train = scaler_.transform(dataset_.train['features']).astype('float32')
    scaled_features_test = scaler_.transform(dataset_.test['features']).astype('float32')
    total_periods = int(dataset_.train['observed_survival'].max()) + 1

    censoring_numeric = ~dataset_.train['censoring']*1
    survial_numeric = dataset_.train['observed_survival']

    censoring_numeric_test = ~dataset_.test['censoring']*1
    survial_numeric_test = dataset_.test['observed_survival']
    time_grid = np.linspace(survial_numeric_test.min(), survial_numeric_test.max(), 50)


    for mod_ in [LogisticHazard, PMF, DeepHitSingle, MTLR]: 
        str_pattern = r"([^\.]*)$"
        model_key = re.search(str_pattern, str(mod_)).group(0)[:-2]

        labtrans = mod_.label_transform(total_periods)
        y_train_ = labtrans.fit_transform(*(survial_numeric, censoring_numeric))

        in_features = scaled_features_train.shape[1]
        out_features = labtrans.out_features

        net = torch.nn.Sequential(
            torch.nn.Linear(in_features, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, 12),
            torch.nn.ReLU(),

            torch.nn.Linear(12, out_features)
        )
        
        model = mod_(net,
                     tt.optim.Adam(0.01),
                     duration_index=labtrans.cuts)

        batch_size = 128
        epochs = 100
        _ = torch.manual_seed(123)
        log = model.fit(scaled_features_train, y_train_, batch_size, epochs) 
        surv = model.predict_surv_df(scaled_features_test)

        evaluation_results = EvalSurv(surv,
                                      survial_numeric_test,
                                      censoring_numeric_test,
                                      censor_surv='km')
        concordance_idx = evaluation_results.concordance_td('antolini')
        brier_idx = evaluation_results.integrated_brier_score(time_grid) 

        all_results[model_key][dataset_key] = {}
        all_results[model_key][dataset_key]['model'] = model
        all_results[model_key][dataset_key]['predictions'] = surv
        all_results[model_key][dataset_key]['concordance'] = concordance_idx
        all_results[model_key][dataset_key]['brier'] = brier_idx

In [28]:
for model_type in  all_results.keys():
    for data_type in all_results[model_type]:
        print(model_type, data_type, np.round(all_results[model_type][data_type]['brier'],3))

LogisticHazard Nonlinear 0.094
LogisticHazard HeavyCensoring 0.06
PMF Nonlinear 0.092
PMF HeavyCensoring 0.06
DeepHitSingle Nonlinear 0.095
DeepHitSingle HeavyCensoring 0.074
MTLR Nonlinear 0.091
MTLR HeavyCensoring 0.061


In [29]:
for model_type in  all_results.keys():
    for data_type in all_results[model_type]:
        print(model_type, data_type, np.round(all_results[model_type][data_type]['concordance'],3))

LogisticHazard Nonlinear 0.61
LogisticHazard HeavyCensoring 0.803
PMF Nonlinear 0.632
PMF HeavyCensoring 0.806
DeepHitSingle Nonlinear 0.628
DeepHitSingle HeavyCensoring 0.799
MTLR Nonlinear 0.63
MTLR HeavyCensoring 0.776


In [30]:
%%capture
# Set up model, use max observed survival for periods, run a sample model
total_periods = int(nonlinear_data.train['observed_survival'].max())

tf.keras.utils.set_random_seed(123)
deep_model_simple = DeepKaplanMeier(total_periods)
deep_model_simple.compile_model((3, ), [20,20], preoutput=12)

In [31]:
dataset_ = nonlinear_data

scaler_ = StandardScaler()
scaler_.fit(dataset_.train['features'])

scaled_features_train = scaler_.transform(dataset_.train['features']).astype('float32')
scaled_features_test = scaler_.transform(dataset_.test['features']).astype('float32')
total_periods = int(dataset_.train['observed_survival'].max()) + 1

censoring_numeric = dataset_.train['censoring']
survial_numeric = dataset_.train['observed_survival']

censoring_numeric_test = ~dataset_.test['censoring']*1
survial_numeric_test = dataset_.test['observed_survival']
time_grid = np.linspace(survial_numeric_test.min(), survial_numeric_test.max(), 50)

In [32]:
# Fit model
tf.keras.utils.set_random_seed(123)
for repeater_epoch in range(10):
  print(f'...this is epoch {10*repeater_epoch+5}')
  deep_model_simple.fit_model(scaled_features_train,
                              survial_numeric.astype(int), 
                              censoring_numeric,
                              epochs=10, 
                              verbose=False, 
                              batch_size=128
                              )

...this is epoch 5
...this is epoch 15
...this is epoch 25
...this is epoch 35
...this is epoch 45
...this is epoch 55
...this is epoch 65
...this is epoch 75
...this is epoch 85
...this is epoch 95


In [33]:
preds_raw = deep_model_simple.model.predict(scaled_features_test)
score_concordance, score_brier = calculate_preds(preds_raw,
                                                 survial_numeric_test,
                                                 censoring_numeric_test)
  
print(score_concordance, score_brier)

32/32 [==============================] - 1s 3ms/step
0.6377901025921783 0.09087990661832093


In [34]:
%%capture
# Set up model, use max observed survival for periods, run a sample model
total_periods = int(heavy_censored_data.train['observed_survival'].max())

tf.keras.utils.set_random_seed(123)
deep_model_simple = DeepKaplanMeier(total_periods)
deep_model_simple.compile_model((3, ), [20,20,20], preoutput=12)

In [35]:
dataset_ = heavy_censored_data

scaler_ = StandardScaler()
scaler_.fit(dataset_.train['features'])

scaled_features_train = scaler_.transform(dataset_.train['features']).astype('float32')
scaled_features_test = scaler_.transform(dataset_.test['features']).astype('float32')
total_periods = int(dataset_.train['observed_survival'].max()) + 1

censoring_numeric = dataset_.train['censoring']
survial_numeric = dataset_.train['observed_survival']

censoring_numeric_test = ~dataset_.test['censoring']*1
survial_numeric_test = dataset_.test['observed_survival']
time_grid = np.linspace(survial_numeric_test.min(), survial_numeric_test.max(), 50)

In [36]:
# Fit model
tf.keras.utils.set_random_seed(123)
for repeater_epoch in range(10):
  print(f'...this is epoch {10*repeater_epoch+5}')
  deep_model_simple.fit_model(scaled_features_train,
                              survial_numeric.astype(int), 
                              censoring_numeric,
                              epochs=10, 
                              verbose=False, 
                              batch_size=128
                              )

...this is epoch 5
...this is epoch 15
...this is epoch 25
...this is epoch 35
...this is epoch 45
...this is epoch 55
...this is epoch 65
...this is epoch 75
...this is epoch 85
...this is epoch 95


In [37]:
preds_raw = deep_model_simple.model.predict(scaled_features_test)
score_concordance, score_brier = calculate_preds(preds_raw,
                                                 survial_numeric_test,
                                                 censoring_numeric_test)
  
print(score_concordance, score_brier)

32/32 [==============================] - 1s 5ms/step
0.8199628441141699 0.05973803489967512
